<a href="https://colab.research.google.com/github/adaltopicotti/imersao-alura-google-desafio-gemini/blob/main/desafio_alura_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consulta de informações em Artigos em PDF

Projeto que auxilia na busca de informações em PDFs.

### Informação relevante para o projeto
 - O pdf utilizado para os testes foi o [gemini-for-google-workspace-prompting-guide-101](https://inthecloud.withgoogle.com/gemini-for-google-workspace-prompt-guide/dl-cd.html).
 - O pdf foi armazenado no diretório raiz do projeto.


 ### Aprimoramento
 - Para este projeto está utilizando apenas 1 PDF, porém o projeto pode ser adaptado para realizar busca em multiplos PDFs.
 - Com a utilização da biblioteca VertexAI, pode ter uma melhora significante na manipulação do PDF, ou até mesmo passar utilizar outros formatos, eliminando a necessidade de bibliotecas de terceiros como a `pypdf` que foi utilizada neste projeto.



In [20]:
!pip install -U -q pypdf
!pip install -U -q google-generativeai

## Importações e configurações iniciais

In [21]:
# Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

from pypdf import PdfReader
import json

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Inicializando modelos

In [130]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.2,
}

# Uma possibilidade: implementar o safety_settings de acordo com o artigo/pdf a ser estudado
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

# Modelo para validações
model_validator = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings,
    system_instruction=[
        "Caso o CONTENT seja um sumário e o INPUT não pedir sumário, responda apenas True. Senão responda False"
    ]
)

# Modelo que gera resposta
model_generative = genai.GenerativeModel(
    model_name="gemini-1.0-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

Obs: o model_validator foi criado apenas para testar como realizar validações utilizando `system_instructions` do modelo `gemini-1.5-pro`.

O mantive no código para quem tiver interesse ou dúvida de como utilizar.

## Funções

In [129]:
def embed_fn(text, model):
  return genai.embed_content(
    model=model,
    content=text,
    task_type="RETRIEVAL_DOCUMENT"
  )['embedding']


In [124]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                content=consulta,
                                task_type="RETRIEVAL_QUERY"
                              )["embedding"]

  produtos_escalares = np.dot(np.stack(base["embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)

  return base.iloc[indice]["content"]

In [125]:
def gerar_dataframe_de_pdf(pdf_path, model):
  inputpdf = PdfReader(open(pdf_path, 'rb'))

  inputpdf.pages[3].extract_text()
  pages = []
  for index, _ in enumerate(inputpdf.pages):
    if index >= 1:
      page_before = inputpdf.pages[index - 1].extract_text()
    else:
      page_before = ""

    if index <= len(inputpdf.pages) - 2:
      page_after = inputpdf.pages[index + 1].extract_text()
    else:
      page_after = ""


    page = inputpdf.pages[index].extract_text()

    pages.append([f"{page_before} \n{page} \n{page_after}"])

  df = pd.DataFrame(pages, columns=["content"])
  df["embeddings"] = df.apply(lambda row: embed_fn(row["content"], model=model), axis=1)
  return df

Cada linha do DataFrame é criado com até 3 páginas do PDF.

Exemplos:
- Para o embedding da página 1, é utilizado a página 1 e 2.
- Para o embedding da página 4, é utilizado a página 3, 4 e 5.

Sempre utilizando a página atual, a anterior (caso exista) e a posterior (caso exista).

## Gerando o DataFrame do PDF `df_pdf`

In [127]:
model_embedding = "models/embedding-001"
df_pdf = gerar_dataframe_de_pdf('gemini-for-google-workspace-prompting-guide-101.pdf', model=model_embedding)

# Mostrar os 5 primeiros items do DataFrame
df_pdf.head(5)

,content,embeddings
0,\n1Prompting\nguide 101\nApril 2024 editionA ...,"[0.045477975, -0.06524749, -0.010696349, -0.02..."
1,1Prompting\nguide 101\nApril 2024 editionA qui...,"[0.038546376, -0.06046804, -0.0048366333, -0.0..."
2,2Writing effective prompts \nFrom the very beg...,"[0.03361882, -0.067133844, -0.006784832, -0.02..."
3,3Prompting is an art. You will likely need to ...,"[0.039101925, -0.06972493, -0.01347802, -0.005..."
4,4Table of contents\nWriting effective prompts ...,"[0.039370213, -0.0792449, -0.03829455, -0.0225..."


## Gerando e buscando a consulta

In [128]:
consulta = "Como escrever prompts efetivos?"

resposta_consulta = gerar_e_buscar_consulta(consulta, df_pdf,model_embedding)

## Utilizando o modelo generativo para uma resposta mais legível em cima da consulta

In [131]:
response = model_generative.generate_content(
    f"SEARCH_RESPONSE: {resposta_consulta} \n Com base no SEARCH_RESPONSE responda: {consulta}")

print(response.text)

**Como escrever prompts efetivos**

Ao escrever um prompt efetivo, considere as seguintes áreas:

* **Persona:** Para quem você está escrevendo?
* **Tarefa:** O que você quer que o AI faça?
* **Contexto:** Forneça informações relevantes para a tarefa.
* **Formato:** Use linguagem natural, seja específico e conciso.

**Dicas:**

* Use linguagem natural e frases completas.
* Seja específico e forneça contexto.
* Seja conciso e evite complexidade.
* Faça uma conversa e refine seus prompts.
* Os prompts ideais têm cerca de 21 palavras.
* Revise as saídas do AI para garantir clareza e precisão.
